# Tindar - The Pairing Problem

Welcome to Tindar: the new app for dating Data Scientists! Through your app, your users have labeled all the other users as either 'interested' (1) or 'not interested' (0). Your job is to make your community happy by pairing up as many users as you can.

Rules:
- pairing can only happen if both Data Scientists marked each other as 'interested'
- your app is conservative, so no threesomes, only pairs
- lonely love doesn't count, you can not pair someone with himself

Hints:
- it's okay if some Data Scientists stay lonely. Most likely, there is not solution that matches everyone.
- think of adjacency matrix

Assumption:
- for simplicity, let's assume that the labels are symmetric. If person A marked person B as interesting, then the reverse holds as well


## Solution

This program can be expressed as a binary linear problem:
"description here"

The nice thing about these problems is that they can be solved optimally (under certain conditions), yielding an uncontestable solution!

You don't have to solve these problems by hand (e.g. by solving KKT conditions and checking some assumptions): all you need is to code them, and let the computer do the work! In python, there is a nice library called PuLP: https://coin-or.github.io/pulp/. We also use numpy to make our lives a bit easier.

Let the love games commence!

In [1]:
from pulp import *
import numpy as np

In [2]:
# Example 1
A = np.array([
    [0, 1, 1, 1],
    [1, 0, 0, 0],
    [1, 0, 0, 1],
    [1, 0, 1, 0]
])

for i in range(A.shape[0]):
    A[i, i] = 0

A

# Hint: Draw the graph, and find the optimal solution for yourself.
# Is this solution unique?

array([[0, 1, 1, 1],
       [1, 0, 0, 0],
       [1, 0, 0, 1],
       [1, 0, 1, 0]])

In [3]:
m, n = A.shape
if m != n:
    raise ValueError(f"A is not square: A.shape = {A.shape}")

In [4]:
x_names = [f"x_{i}{j}" for i in range(m) for j in range(n)]
print(x_names)

['x_00', 'x_01', 'x_02', 'x_03', 'x_10', 'x_11', 'x_12', 'x_13', 'x_20', 'x_21', 'x_22', 'x_23', 'x_30', 'x_31', 'x_32', 'x_33']


In [5]:
x = [LpVariable(name=x_name, cat="Binary") for x_name in x_names]
print(x[0])
print(type(x[0]))

x_00
<class 'pulp.pulp.LpVariable'>


In [6]:
x_np = np.array(x).reshape((n, n))
x_np

array([[x_00, x_01, x_02, x_03],
       [x_10, x_11, x_12, x_13],
       [x_20, x_21, x_22, x_23],
       [x_30, x_31, x_32, x_33]], dtype=object)

## symmetry constraints

In [7]:
# lhs: Left Hand Side
lhs_symmetry = [
    LpAffineExpression([(x_np[i,j], 1), (x_np[j,i], -1)], name=f"lhs_sym_{i}{j}")
    for i in range(n) for j in range(i+1, n)
]

In [8]:
constraints_symmetry = [
    LpConstraint(
        e=lhs_s,
        sense=0,
        name=f"constraint_sym_{lhs_s.name[-2:]}",
        rhs=0
    )
    for lhs_s in lhs_symmetry
]

if len(constraints_symmetry) != (n**2-n)/2:
    raise Exception(
        "Symmetry constraints not constructed right:"
        f"A.shape = {A.shape}, len(constraints_symmetry) should be "
        f"{(n**2-n)/2}, actually is {len(constraints_symmetry)}"
    )

## only assignable if student likes the other

In [9]:
lhs_like = [
    LpAffineExpression([(x_np[i, j], 1)], name=f"lhs_like_{i}{j}")
    for i in range(n) for j in range(n)
]

In [10]:
constraints_like = [
    LpConstraint(
        e=lhs_l,
        sense=-1,
        name=f"constraint_like_{lhs_l.name[-2:]}",
        rhs=A[int(lhs_l.name[-2]), int(lhs_l.name[-1])]
    )
    for lhs_l in lhs_like
]
constraints_like[0].name

if len(constraints_like) != n**2:
    raise Exception(
        "Liking constraints not constructed right:"
        f"A.shape = {A.shape}, len(constraints_like) should be "
        f"{n**2}, actually is {len(constraints_like)}"
    )

## One person can be asigned to only one other person

In [11]:
def make_single_constraints(lhs_single, kind):
    constraints_single = [
        LpConstraint(
            e=lhs_s,
            sense=-1,
            name=f"constraint_single_{kind}_{lhs_s.name[-2:]}",
            rhs=1
        )
        for lhs_s in lhs_single
    ]
    
    return constraints_single  
    

lhs_single_rowsum = [
    LpAffineExpression(
        [(x_np[i, j], 1) for j in range(n)],
        name=f"lhs_single_rowsum_{i}*"
    )
    for i in range(n)
]

lhs_single_colsum = [
    LpAffineExpression(
        [(x_np[i, j], 1) for i in range(n)],
        name=f"lhs_single_colsum_{j}*"
    )
    for j in range(n)
]

constraints_single_rowsum = make_single_constraints(lhs_single_rowsum, "rowsum")
constraints_single_colsum = make_single_constraints(lhs_single_colsum, "colsum")


def check_single_constraints(constraints_single, kind):
    if len(constraints_single) != n:
        raise Exception(
            f"Constraints single {kind} not constructed right:"
            f"A.shape = {A.shape}, len(constraints_single_{kind}) should be "
            f"{n}, actually is {len(constraints_single)}"
        )

check_single_constraints(constraints_single_rowsum, "rowsum")
check_single_constraints(constraints_single_colsum, "colsum")

## merge constraints

In [12]:
constraints_all = [
    *constraints_symmetry,
    *constraints_like,
    *constraints_single_rowsum,
    *constraints_single_colsum
]
constraints_all

[1*x_01 + -1*x_10 + 0 = 0,
 1*x_02 + -1*x_20 + 0 = 0,
 1*x_03 + -1*x_30 + 0 = 0,
 1*x_12 + -1*x_21 + 0 = 0,
 1*x_13 + -1*x_31 + 0 = 0,
 1*x_23 + -1*x_32 + 0 = 0,
 1*x_00 + 0 <= 0,
 1*x_01 + -1 <= 0,
 1*x_02 + -1 <= 0,
 1*x_03 + -1 <= 0,
 1*x_10 + -1 <= 0,
 1*x_11 + 0 <= 0,
 1*x_12 + 0 <= 0,
 1*x_13 + 0 <= 0,
 1*x_20 + -1 <= 0,
 1*x_21 + 0 <= 0,
 1*x_22 + 0 <= 0,
 1*x_23 + -1 <= 0,
 1*x_30 + -1 <= 0,
 1*x_31 + 0 <= 0,
 1*x_32 + -1 <= 0,
 1*x_33 + 0 <= 0,
 1*x_00 + 1*x_01 + 1*x_02 + 1*x_03 + -1 <= 0,
 1*x_10 + 1*x_11 + 1*x_12 + 1*x_13 + -1 <= 0,
 1*x_20 + 1*x_21 + 1*x_22 + 1*x_23 + -1 <= 0,
 1*x_30 + 1*x_31 + 1*x_32 + 1*x_33 + -1 <= 0,
 1*x_00 + 1*x_10 + 1*x_20 + 1*x_30 + -1 <= 0,
 1*x_01 + 1*x_11 + 1*x_21 + 1*x_31 + -1 <= 0,
 1*x_02 + 1*x_12 + 1*x_22 + 1*x_32 + -1 <= 0,
 1*x_03 + 1*x_13 + 1*x_23 + 1*x_33 + -1 <= 0]

## objective: make as many matches as possible

In [13]:
objective = LpAffineExpression([(x_i, 1) for x_i in x])
objective

1*x_00 + 1*x_01 + 1*x_02 + 1*x_03 + 1*x_10 + 1*x_11 + 1*x_12 + 1*x_13 + 1*x_20 + 1*x_21 + 1*x_22 + 1*x_23 + 1*x_30 + 1*x_31 + 1*x_32 + 1*x_33 + 0

## define full problem

In [14]:
prob = LpProblem("The Tindar Problem", LpMaximize)
prob += objective

for c in constraints_all:
    prob += c

In [15]:
prob.writeLP("../models/Tindar.lp")

## solution

In [16]:
prob.solve()

1

In [17]:
print("Status:", LpStatus[prob.status])

Status: Optimal


In [18]:
for v in prob.variables():
    print(v.name, "=", v.varValue)

x_00 = 0.0
x_01 = 1.0
x_02 = 0.0
x_03 = 0.0
x_10 = 1.0
x_11 = 0.0
x_12 = 0.0
x_13 = 0.0
x_20 = 0.0
x_21 = 0.0
x_22 = 0.0
x_23 = 1.0
x_30 = 0.0
x_31 = 0.0
x_32 = 1.0
x_33 = 0.0


In [19]:
print("Number of lovers connected = ", value(prob.objective))

Number of lovers connected =  4.0


## Q1: what happens to the model when you drop the symmetry of interested people?

## Q2: what if people did not express their interest as binary, but as an integer (let's say 1-5). How could you change the model to make the community as happy as possible?